# Quality Control

In [ ]:
import numpy as np
import scanpy as sc
import seaborn as sns
import pandas as pd
from scipy.stats import median_abs_deviation
import anndata2ri
import logging
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro
import os
os.chdir("/data/home/wx/") 
rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=80,
    facecolor="white",
    frameon=False,
)


In [ ]:
%%R
inputdir<-c('scislets/data/CT/output/')
outputdir<-c('scislets/data/CT/output/h5ad')
library(Seurat)
library(MuDataSeurat)
data <- Read10X(data.dir = paste0(inputdir,'/','filter_matrix'), gene.colum=1)
data <- CreateSeuratObject(counts = data, min.cells = 0, min.features = 0, project = "CT")
MuDataSeurat::WriteH5AD(data, paste0(outputdir,'/','filter.h5ad'), assay="RNA")
data.raw<-Read10X(data.dir = paste0(inputdir,'/','raw_matrix'), gene.colum=1)
data.raw <- CreateSeuratObject(counts = data.raw, min.cells = 0, min.features = 0, project = "CT")
MuDataSeurat::WriteH5AD(data.raw, paste0(outputdir,'/','raw.h5ad'), assay="RNA")

In [ ]:
adata = sc.read_h5ad(
    filename='/data/home/wx/scislets/data/CT/output/h5ad/filter.h5ad'
)
adata

In [ ]:
adata.var_names_make_unique()

In [ ]:
# mitochondrial genes
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes.
adata.var["hb"] = adata.var_names.str.contains(("^HB[^(P)]"))

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True
)

In [ ]:
#Basic filtering: 
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
p1 = sns.displot(adata.obs["total_counts"], bins=100, kde=False)
p2 = sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],jitter=0.4, multi_panel=True)
p3 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

In [ ]:
#Basic filtering:
adata = adata[adata.obs.n_genes_by_counts < 11000, :]
adata = adata[adata.obs.n_genes_by_counts > 1500, :]
#adata = adata[adata.obs.total_counts > 2000, :]

In [ ]:
p1 = sns.displot(adata.obs["total_counts"], bins=100, kde=False)
p2 = sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts'],jitter=0.4, multi_panel=True)
p3 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

In [ ]:
#filter
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", 5)
    | is_outlier(adata, "log1p_n_genes_by_counts", 5)
    | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
)
adata.obs.outlier.value_counts()

In [ ]:
adata.obs["mt_outlier"] = is_outlier(adata, "pct_counts_mt", 3) | (
    adata.obs["pct_counts_mt"] > 8
)
adata.obs.mt_outlier.value_counts()

In [ ]:
print(f"Total number of cells: {adata.n_obs}")
adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()

print(f"Number of cells after filtering of low quality cells: {adata.n_obs}")

In [ ]:
p1 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

## Correction of ambient RNA

In [ ]:
%%R
library(SoupX)

In [ ]:
adata_pp = adata.copy()
sc.pp.normalize_per_cell(adata_pp)
sc.pp.log1p(adata_pp)

In [ ]:
sc.pp.pca(adata_pp)
sc.pp.neighbors(adata_pp)
sc.tl.leiden(adata_pp, key_added="soupx_groups")

# Preprocess variables for SoupX
soupx_groups = adata_pp.obs["soupx_groups"]

In [ ]:
del adata_pp

In [ ]:
cells = adata.obs_names
genes = adata.var_names
data = adata.X.T

In [ ]:
data

In [ ]:
adata_raw = sc.read_h5ad(
    filename="/data/home/wx/scislets/data/CT/output/h5ad/raw.h5ad"
)
adata_raw.var_names_make_unique()
cells_raw = adata_raw.obs_names
genes_raw = adata_raw.var_names
data_tod = adata_raw.X.T


In [ ]:
del adata_raw

In [ ]:
%%R -i data -i data_tod -i genes -i cells  -i genes_raw -i cells_raw -i soupx_groups -o out

# specify row and column names of data
rownames(data) = genes
colnames(data) = cells
rownames(data_tod) = genes_raw
colnames(data_tod) = cells_raw
data_tod <- data_tod[rownames(data),]
# ensure correct sparse format for table of counts and table of droplets
data <- as(data, "sparseMatrix")
data_tod <- as(data_tod, "sparseMatrix")
#data_tod <- subset(data_tod,rownames(data_tod) %in% rownames(data))
# Generate SoupChannel Object for SoupX 
sc = SoupChannel(data_tod, data, calcSoupProfile = FALSE)

# Add extra meta data to the SoupChannel object
soupProf = data.frame(row.names = rownames(data), est = rowSums(data)/sum(data), counts = rowSums(data))
sc = setSoupProfile(sc, soupProf)
# Set cluster information in SoupChannel
sc = setClusters(sc, soupx_groups)

# Estimate contamination fraction
sc  = autoEstCont(sc, doPlot=FALSE)
# Infer corrected table of counts and rount to integer
out = adjustCounts(sc, roundToInt = TRUE)

In [ ]:
adata.layers["counts"] = adata.X
adata.layers["soupX_counts"] = out.T
adata.X = adata.layers["soupX_counts"]

In [ ]:
print(f"Total number of genes: {adata.n_vars}")

# Min 20 cells - filters out 0 count genes
sc.pp.filter_genes(adata, min_cells=20)
print(f"Number of genes after cell filter: {adata.n_vars}")

(rna:doublet-detection)=
## Doublet Detection

In [ ]:
%%R
library(Seurat)
library(scater)
library(scDblFinder)
library(BiocParallel)

In [ ]:
data_mat = adata.X.T

In [ ]:
%%R -i data_mat -o doublet_score -o doublet_class

set.seed(123)
sce = scDblFinder(
    SingleCellExperiment(
        list(counts=data_mat),
    ) 
)
doublet_score = sce$scDblFinder.score
doublet_class = sce$scDblFinder.class

In [ ]:
adata.obs["scDblFinder_score"] = doublet_score
adata.obs["scDblFinder_class"] = doublet_class
adata.obs.scDblFinder_class.value_counts()

In [ ]:
adata=adata[adata.obs.scDblFinder_class=="singlet", :]

In [ ]:
adata.obs.scDblFinder_class

In [ ]:
adata.write("scislets/processed/CT_quality_control.h5ad")